# IonQ Boostvqe demo

In this notebook, we demonstrate how to run boostvqe on IonQ emulators (similarly hardwarws with some backend modifications).

We are following on the DBQA GCI circuit synthesis notebook, which can be found following the link: https://github.com/qiboteam/boostvqe/blob/quantinuum_boostvqe-demo/notebooks/quantinuum_demo/gci_boostvqe_circuit_synthesis.ipynb

It supplies a VQE and a GCI circuits in QASM format which we will load our circuits from.

## Step 1: Load circuits

In [16]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qibo import hamiltonians
import matplotlib.pyplot as plt
from ionq_utils import *
from scipy.linalg import eigvalsh

In [17]:
# file paths
vqe_circ_file = "results/vqe_circ_4.qasm"
gci_circ_file = "results/gci_circ_4.qasm"

In [18]:
vqe_circ = QuantumCircuit.from_qasm_file(vqe_circ_file)
# render_circuit_jupyter(vqe_circ)

In [19]:
gci_circ = QuantumCircuit.from_qasm_file(gci_circ_file)
# render_circuit_jupyter(gci_circ)

### Check analytical energy

In this step we are making sure that the correct circuits have been imported.

In [20]:
# create target hamiltonian in qiskit
ham = xxz_hamiltonian(4, 0.5)
ham_matrix = ham.to_matrix()
target_energy = min(eigvalsh(ham_matrix))

In [21]:
state_vec_vqe = Statevector.from_instruction(vqe_circ)
expectation_value_vqe = np.real(np.dot(state_vec_vqe.data.conj().T, np.dot(ham_matrix, state_vec_vqe.data)))
state_vec_gci = Statevector.from_instruction(gci_circ)
expectation_value_gci = np.real(np.dot(state_vec_gci.data.conj().T, np.dot(ham_matrix, state_vec_gci.data)))
print("Target energy:", target_energy)
print("VQE energy:", expectation_value_vqe)
print("GCI energy:", expectation_value_gci)

Target energy: -6.74456264653803
VQE energy: -6.309812021233839
GCI energy: -6.741347828261663


## Step 2: Connect to IonQ server

In [22]:
from qiskit_ionq import IonQProvider
from copy import deepcopy

my_api_key = "pOiUVlzriOoF2wX1kp3lIqid1OMhwXZ5"
provider = IonQProvider(my_api_key)
ionq_backend = provider.get_backend("ionq_simulator")

/Users/pethidine/Documents/GitHub/boostvqe/.conda/lib/python3.11/site-packages/qiskit_ionq/helpers.py:563: UserWarning: Unable to get qubit count for simulator: 'qubits'. Defaulting to 100.
  warnings.warn(
/Users/pethidine/Documents/GitHub/boostvqe/.conda/lib/python3.11/site-packages/qiskit_ionq/helpers.py:563: UserWarning: Unable to get qubit count for ionq_qpu: 'qubits'. Defaulting to 100.
  warnings.warn(
/Users/pethidine/Documents/GitHub/boostvqe/.conda/lib/python3.11/site-packages/qiskit_ionq/helpers.py:563: UserWarning: Unable to get qubit count for ionq_simulator: 'qubits'. Defaulting to 100.
  warnings.warn(


## Step 3: Prepare measurement circuits

In this step, we prepare a list of measurement circuits to measure the energy expectation of the Hamiltonian.
Since our example uses XXZ model, we will need to measure the circuits in 3 separate bases:
1. X
2. Y
3. Z

In [ ]:
from pytket.extensions.qiskit import tk_to_qiskit
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit import Qubit
from pytket.utils.operators import QubitPauliOperator

In [ ]:
meas_vqe_circ_list = rotate_circuit_XYZ(vqe_circ)
meas_gci_circ_list = rotate_circuit_XYZ(gci_circ)

## Step 4: Define loss function
We define a function to convert the measurement results into the expectation energy.

In [43]:
shots = 1000
observable_labels = ['XX', 'YY', 'ZZ']
xxz_expectation = 0
for i,qc_m in enumerate(meas_vqe_circ_list):
    job = ionq_backend.run(qc_m, shots=shots)
    result = job.result()
    distribution = result.get_probabilities()
    expectation = sample_to_expectation(ham_matrix, distribution)
    xxz_expectation += expectation
    print(observable_labels[i], expectation)
    
print('Expectation energy:', xxz_expectation)

NameError: name 'meas_vqe_circ_list' is not defined

In [ ]:
shots = 1000
observable_labels = ['XX', 'YY', 'ZZ']
xxz_expectation = 0
for i,qc_m in enumerate(meas_gci_circ_list):
    job = ionq_backend.run(qc_m, shots=shots)
    result = job.result()
    distribution = result.get_probabilities()
    expectation = sample_to_expectation(ham_matrix, distribution)
    xxz_expectation += expectation
    print(observable_labels[i], expectation)
    
print('Expectation energy:', xxz_expectation)

XX -1.3897576900000002
YY -1.3910324479999998
ZZ -1.180009208
Expectation energy: -3.960799346


In [ ]:
nqubits = 4
delta = 0.5
xx = xxz_hamiltonian(nqubits,delta,select='XX')
yy = xxz_hamiltonian(nqubits,delta,select='YY')
zz = xxz_hamiltonian(nqubits,delta,select='ZZ')
coefs = [1,1,0.5]
observables = [xx,yy,zz]
for i, ob in enumerate(observables):
    expectation = coefs[i]*np.real(np.dot(state_vec_gci.data.conj().T, np.dot(ob.to_matrix(), state_vec_gci.data)))
    print(expectation)

-2.7795153768806555
-2.782064895225319
-0.5898837780778438


In [ ]:
shots = 1000
keys_ls = []
frequencies_ls = []
observable_labels = ['XX', 'YY', 'ZZ']
xxz_expectation = 0
for i,qc_m in enumerate(meas_gci_circ_list):
    job = ionq_backend.run(qc_m, shots=shots)
    result = job.result()
    # Get the counts from the result
    counts = result.get_counts()
    keys = [key.split()[0][::-1] for key in counts.keys()]
    sample_counts = list(counts.values())
    frequencies = np.array(sample_counts) / sum(sample_counts)
    ob = zz
    coef = coefs[i]
    dict1 = dict(zip(keys, frequencies))
    expectation = sample_to_expectation(ob.to_matrix(), dict1)*coef
    xxz_expectation += expectation
    print(observable_labels[i], expectation)
    keys_ls.append(keys)
    frequencies_ls.append(frequencies)
    
print('Expectation energy:', xxz_expectation)

XX -1.422
YY -1.3499999999999999
ZZ -0.589
Expectation energy: -3.3609999999999998
